In [1]:
import xarray
import math
import numpy as np
from data_to_xarray import coords_to_xarray

In [ ]:
coords_to_xarray('gene_coords', 'gene_expression_coords_dataset.nc',masked=False, pad_mode='repeat',
                           num_points=200, normalize=False)

Select only specific patterns

In [2]:
dataset = xarray.load_dataset('/media/gambino/students_workdir/ilias/all_simulated_data/gene_expression_coords_masked_dataset.nc')

Rotation

In [10]:
rot_augmented_dataset = dataset.copy()

for i in range(3):
    angle=math.radians(np.random.randint(0,180))
    x = dataset.trans_coords[:,:2].values
    centroids = np.expand_dims(x.sum(2)/len(dataset.spots), axis=2)

    dataset_copy = dataset.copy()
    dataset_copy.trans_coords[:,:2].values = np.stack((centroids[:,0] + math.cos(angle) * (x[:,0] - centroids[:,0]) - math.sin(angle) * (x[:,1] - centroids[:,1]),
                centroids[:,1] + math.sin(angle) * (x[:,0] - centroids[:,0]) + math.cos(angle) * (x[:,1] - centroids[:,1])),axis=1)
    rot_augmented_dataset = xarray.concat((rot_augmented_dataset, dataset_copy), dim='index')

# rot_augmented_dataset.to_netcdf('gene_expression_coords_masked_rotated_dataset.nc',engine='netcdf4')

Selected patterns

In [11]:
selected_patterns = ['cell_edge', 'foci', 'extranuclear', 'intranuclear', 'pericellular', 'perinuclear', 'random', 'protrusion']
dataset = dataset.isel(index = dataset['pattern'].isin(selected_patterns))
# dataset.to_netcdf('gene_expression_coords_masked_reduced_dataset.nc',engine='netcdf4')

Exclude cell identities

In [14]:
cell_ids = [metadata.split('_')[4] if metadata.split('_')[1]=='edge' else metadata.split('_')[3] for metadata in dataset.metadata.to_dataframe().iloc[:,0]]
indices_to_include = [i for i, cellid in enumerate(cell_ids) if cellid in ['icell' + str(i) for i in range(250)]]
exclude_cellids_dataset = dataset.isel(dict(index=indices_to_include))
# exclude_cellids_dataset.to_netcdf('gene_expression_coords_masked_cellids-excluded_dataset.nc',engine='netcdf4')